# OHCL - Analytics

### Import Library

In [69]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [70]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [71]:
data = dataset.copy()

In [72]:
data["ema_f2"] = ta.ema(data["Close"], length=16)
data["ema_l2"] = ta.ema(data["Close"], length=250)
data["long_trend"] = data.apply(lambda r: 'up' if r['ema_f2'] > r['ema_l2'] else 'down', axis=1)
data.drop(columns=['ema_f2', 'ema_l2'], inplace=True)
data.dropna(inplace=True)

In [73]:
def cal_fibonacci(htd):
    _1_d_df = htd.resample("D").agg({
        'Open': 'first',
        'Close': 'last',
        'High': 'max',
        'Low': 'min',
        'long_trend': 'first'
    })
    _1_d_df.dropna(inplace=True)
    _1_d_df["high_low"] = _1_d_df.apply(lambda r: r['High'] - r['Low'], axis=1)
    _1_d_df["oc"] = _1_d_df.apply(lambda r: r['Open'] - r['Close'], axis=1)
    _1_d_df["F1"] = _1_d_df.apply(lambda r: r['Low'] if r['oc'] < 0 else r['High'], axis=1)
    _1_d_df["F7"] = _1_d_df.apply(lambda r: r['High'] if r['oc'] < 0 else r['Low'], axis=1)
    _1_d_df["F2"] = _1_d_df.apply(
        lambda r: round(r['F1'] + 0.236 * r['high_low'], 1) if r['oc'] < 0 else round(r['F1'] - 0.236 * r['high_low'],
                                                                                      1), axis=1)
    _1_d_df["F3"] = _1_d_df.apply(
        lambda r: round(r['F1'] + 0.382 * r["high_low"], 1) if r['oc'] < 0 else round(r['F1'] - 0.382 * r['high_low'],
                                                                                      1), axis=1)
    _1_d_df["F4"] = _1_d_df.apply(
        lambda r: round(r['F1'] + 0.5 * r["high_low"], 1) if r['oc'] < 0 else round(r['F1'] - 0.5 * r['high_low'], 1),
        axis=1)
    _1_d_df["F5"] = _1_d_df.apply(
        lambda r: round(r['F1'] + 0.618 * r["high_low"], 1) if r['oc'] < 0 else round(r['F1'] - 0.618 * r['high_low'],
                                                                                      1), axis=1)
    _1_d_df["F6"] = _1_d_df.apply(
        lambda r: round(r['F1'] + 0.786 * r["high_low"], 1) if r['oc'] < 0 else round(r['F1'] - 0.786 * r['high_low'],
                                                                                      1), axis=1)
    _1_d_df["F1"] = _1_d_df["F1"].shift(1)
    _1_d_df["F2"] = _1_d_df["F2"].shift(1)
    _1_d_df["F3"] = _1_d_df["F3"].shift(1)
    _1_d_df["F4"] = _1_d_df["F4"].shift(1)
    _1_d_df["F5"] = _1_d_df["F5"].shift(1)
    _1_d_df["F6"] = _1_d_df["F6"].shift(1)
    _1_d_df["F7"] = _1_d_df["F7"].shift(1)
    _1_d_df = _1_d_df[['Open', 'Close', 'High', 'Low', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'long_trend']]
    _1_d_df.dropna(inplace=True)
    return _1_d_df

In [74]:
_1_d_df = cal_fibonacci(data)
_1_d_df

,Open,Close,High,Low,F1,F2,F3,F4,F5,F6,F7,long_trend
Date,,,,,,,,,,,,
2018-08-14,955.5,959.3,961.8,952.1,942.3,945.2,947.0,948.5,949.9,952.0,954.6,down
2018-08-15,958.0,946.6,962.3,946.6,952.1,954.4,955.8,957.0,958.1,959.7,961.8,down
2018-08-16,942.4,947.0,947.3,935.5,962.3,958.6,956.3,954.5,952.6,950.0,946.6,down
2018-08-17,952.2,947.0,954.6,945.0,935.5,938.3,940.0,941.4,942.8,944.8,947.3,down
2018-08-20,946.1,945.0,951.5,942.6,954.6,952.3,950.9,949.8,948.7,947.1,945.0,down
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-29,1321.1,1326.8,1330.1,1320.6,1317.0,1318.8,1319.9,1320.8,1321.7,1323.0,1324.6,up
2024-08-30,1328.4,1332.7,1334.9,1328.0,1320.6,1322.8,1324.2,1325.3,1326.5,1328.1,1330.1,up
2024-09-04,1320.8,1314.0,1321.5,1307.3,1328.0,1329.6,1330.6,1331.5,1332.3,1333.4,1334.9,up


In [75]:
def get_high_zone(r):
    zone = 0
    if r['F1'] < r['F7'] < 0:
        if r['High'] < r['F1']:
            zone = -1
        elif r['F1'] <= r['High'] < r['F2'] :
            zone = 1
        elif r['F2'] <= r['High'] < r['F3'] :
            zone = 2
        elif r['F3'] <= r['High'] < r['F4'] :
            zone = 3
        elif r['F4'] <= r['High'] < r['F5'] :
            zone = 4
        elif r['F5'] <= r['High'] < r['F6'] :
            zone = 5
        elif r['F6'] <= r['High'] < r['F7'] :
            zone = 6
        else:
            zone = 7
    else:
        if r['High'] < r['F7']:
            zone = -1
        elif r['F7'] <= r['High'] < r['F6'] :
            zone = 1
        elif r['F6'] <= r['High'] < r['F5'] :
            zone = 2
        elif r['F5'] <= r['High'] < r['F4'] :
            zone = 3
        elif r['F4'] <= r['High'] < r['F3'] :
            zone = 4
        elif r['F3'] <= r['High'] < r['F2'] :
            zone = 5
        elif r['F2'] <= r['High'] < r['F1'] :
            zone = 6
        else:
            zone = 7
    return zone

In [76]:
_1_d_df['high_zone'] = _1_d_df.apply(lambda r: get_high_zone(r), axis=1)
# _1_d_df['low_zone'] = _1_d_df.apply(lambda r: get_low_zone(r), axis=1)

In [77]:
up_trend = _1_d_df[_1_d_df['long_trend'] == 'up']
down_trend = _1_d_df[_1_d_df['long_trend'] == 'down']

In [81]:
grpd = down_trend.groupby('high_zone')[['Open']]
irs = grpd.count().reset_index().rename(columns={'Open': 'sum'})
irs

,high_zone,sum
0,-1,128
1,1,35
2,2,43
3,3,29
4,4,53
5,5,45
6,6,68
7,7,269


In [79]:
down_trend

,Open,Close,High,Low,F1,F2,F3,F4,F5,F6,F7,long_trend,high_zone
Date,,,,,,,,,,,,,
2018-08-14,955.5,959.3,961.8,952.1,942.3,945.2,947.0,948.5,949.9,952.0,954.6,down,7
2018-08-15,958.0,946.6,962.3,946.6,952.1,954.4,955.8,957.0,958.1,959.7,961.8,down,7
2018-08-16,942.4,947.0,947.3,935.5,962.3,958.6,956.3,954.5,952.6,950.0,946.6,down,1
2018-08-17,952.2,947.0,954.6,945.0,935.5,938.3,940.0,941.4,942.8,944.8,947.3,down,7
2018-08-20,946.1,945.0,951.5,942.6,954.6,952.3,950.9,949.8,948.7,947.1,945.0,down,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-07,1252.3,1250.0,1255.5,1245.7,1237.2,1241.6,1244.4,1246.6,1248.8,1252.0,1256.0,down,-1
2024-08-08,1248.4,1240.1,1251.3,1237.4,1255.5,1253.2,1251.8,1250.6,1249.4,1247.8,1245.7,down,4
2024-08-09,1249.1,1264.9,1266.2,1247.7,1251.3,1248.0,1246.0,1244.3,1242.7,1240.4,1237.4,down,7
